In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
# Generate synthetic regression data
X, y_reg = make_regression(n_samples=1000, n_features=10, noise=0.1)
y_class = (y_reg > y_reg.mean()).astype(int)  # Binary classification label

In [3]:
# Train-test split 
X_train, X_test, y_reg_train, y_reg_test, y_class_train, y_class_test = train_test_split(X, y_reg, y_class, test_size = 0.2, random_state=42)

In [4]:
# Normalize the features 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)  
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_reg_train_tensor = torch.tensor(y_reg_train, dtype=torch.float32).view(-1, 1)
y_reg_test_tensor = torch.tensor(y_reg_test, dtype=torch.float32).view(-1, 1)
y_class_train_tensor = torch.tensor(y_class_train, dtype=torch.float32).view(-1, 1)
y_class_test_tensor = torch.tensor(y_class_test, dtype=torch.float32).view(-1, 1)

In [5]:
# model building
class MultiTaskNet(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.regression_head = nn.Linear(32, 1)
        self.classification_head = nn.Sequential(
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        shared_out = self.shared(x)
        reg_out = self.regression_head(shared_out)
        class_out = self.classification_head(shared_out)
        return reg_out, class_out


In [6]:
# model training and instatiation
model = MultiTaskNet(input_size=X_train_tensor.shape[1])
criterion_reg = nn.MSELoss()
critreion_class = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [14]:
# Training Loop 
num_epochs = 10000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    reg_out, class_out = model(X_train_tensor)
    
    # Compute losses
    loss_reg = criterion_reg(reg_out, y_reg_train_tensor)
    loss_class = critreion_class(class_out, y_class_train_tensor)
    
    # Total loss
    loss = loss_reg + loss_class
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/10000], Loss: 56.8785
Epoch [20/10000], Loss: 55.9209
Epoch [30/10000], Loss: 54.9741
Epoch [40/10000], Loss: 54.0382
Epoch [50/10000], Loss: 53.1139
Epoch [60/10000], Loss: 52.1986
Epoch [70/10000], Loss: 51.2918
Epoch [80/10000], Loss: 50.3972
Epoch [90/10000], Loss: 49.5123
Epoch [100/10000], Loss: 48.6395
Epoch [110/10000], Loss: 47.7825
Epoch [120/10000], Loss: 46.9389
Epoch [130/10000], Loss: 46.1090
Epoch [140/10000], Loss: 45.2917
Epoch [150/10000], Loss: 44.4858
Epoch [160/10000], Loss: 43.6918
Epoch [170/10000], Loss: 42.9081
Epoch [180/10000], Loss: 42.1354
Epoch [190/10000], Loss: 41.3760
Epoch [200/10000], Loss: 40.6268
Epoch [210/10000], Loss: 39.8894
Epoch [220/10000], Loss: 39.1636
Epoch [230/10000], Loss: 38.4451
Epoch [240/10000], Loss: 37.7361
Epoch [250/10000], Loss: 37.0387
Epoch [260/10000], Loss: 36.3505
Epoch [270/10000], Loss: 35.6717
Epoch [280/10000], Loss: 35.0028
Epoch [290/10000], Loss: 34.3465
Epoch [300/10000], Loss: 33.7027
Epoch [310/10000], 

In [15]:
model.eval()
with torch.no_grad():
    y_pred_reg, y_pred_class = model(X_test_tensor)
    
    reg_mse = criterion_reg(y_pred_reg, y_reg_test_tensor)
    class_acc = ((y_pred_class > 0.5) == y_class_test_tensor).float().mean()
    
print(f"Test Regression MSE: {reg_mse.item():.4f}")
print(f"Test Classification Accuracy: {class_acc.item():.4f}")


Test Regression MSE: 1.1217
Test Classification Accuracy: 0.9950
